<a href="https://colab.research.google.com/github/smishr97/Mechanism-of-Action-MoA---DeepNN/blob/main/GPU_MoA_Optimizing_NN_(using_Optuna).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install iterative-stratification

**KAGGLE - Download the dataset**

In [2]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c lish-moa
! kaggle config set -n iterativestratification
! unzip sample_submission.csv.zip
! unzip train_features.csv.zip
! unzip train_targets_nonscored.csv.zip
! unzip train_drug.csv
! unzip train_targets_scored.csv.zip
! unzip test_features.csv.zip

  0% 0.00/257k [00:00<?, ?B/s]
100% 257k/257k [00:00<00:00, 35.1MB/s]
  0% 0.00/233k [00:00<?, ?B/s]
100% 233k/233k [00:00<00:00, 70.3MB/s]
 77% 42.0M/54.8M [00:00<00:00, 53.7MB/s]
100% 54.8M/54.8M [00:00<00:00, 86.4MB/s]
  0% 0.00/38.9k [00:00<?, ?B/s]
100% 38.9k/38.9k [00:00<00:00, 34.1MB/s]
  0% 0.00/9.16M [00:00<?, ?B/s]
100% 9.16M/9.16M [00:00<00:00, 83.6MB/s]
  0% 0.00/535k [00:00<?, ?B/s]
100% 535k/535k [00:00<00:00, 176MB/s]
usage: kaggle config set [-h] -n NAME -v VALUE
kaggle config set: error: argument -v/--value is required
Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   
Archive:  train_features.csv.zip
  inflating: train_features.csv      
Archive:  train_targets_nonscored.csv.zip
  inflating: train_targets_nonscored.csv  
Archive:  train_drug.csv
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment w

In [3]:
import pandas as pd
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
if __name__ == "__main__":
    TARGET_PATH = "/content/train_targets_scored.csv"
    df = pd.read_csv(TARGET_PATH)
    df.loc[:, 'kfold'] = -1
    targets = df.drop("sig_id", axis=1).values

    mskf = MultilabelStratifiedKFold(n_splits=5)

    for fold, (train_idx, valid_idx) in enumerate(mskf.split(X=df, y=targets)):
        df.loc[valid_idx, 'kfold'] = fold

    df.to_csv("/content/train_targets_fold.csv", index=False)

In [4]:
import torch
import pandas as pd
import numpy as np

! pip install optuna
import optuna

     |████████████████████████████████| 308 kB 5.3 MB/s 
     |████████████████████████████████| 209 kB 44.2 MB/s 
     |████████████████████████████████| 80 kB 8.8 MB/s 
     |████████████████████████████████| 75 kB 4.0 MB/s 
     |████████████████████████████████| 49 kB 5.4 MB/s 
     |████████████████████████████████| 144 kB 48.0 MB/s 
     |████████████████████████████████| 111 kB 44.4 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=cc00434ad75723ee014507ff981f45fcb6d396d323431d4bdfbd38828072ea0d
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [5]:
import torch
import torch.nn as nn

class MoaDataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, item):
        return {
            "x" : torch.tensor(self.features[item, :], dtype=torch.float32),
            "y": torch.tensor(self.targets[item, :], dtype=torch.float32),
        }

# Engine class is use for evaluating and training
class Engine:
    def __init__(self, model, optimizer, device):
        self.model = model
        self.device = device
        self.optimizer = optimizer

    @staticmethod
    def loss_fn(targets, outputs):
        return nn.BCEWithLogitsLoss()(outputs, targets)

    # Training functions
    def train(self, data_loader):
        self.model.train()
        final_loss = 0
        for data in data_loader:
            self.optimizer.zero_grad()
            inputs = data["x"].to(self.device)
            targets = data["y"].to(self.device)
            outputs = self.model(inputs)
            loss = self.loss_fn(targets, outputs)
            loss.backward()
            self.optimizer.step()
            final_loss += loss.item()
        return final_loss / len(data_loader)

    # Evaluating function
    def evaluate(self, data_loader):
        self.model.eval()
        final_loss = 0
        for data in data_loader:
            inputs = data["x"].to(self.device)
            targets = data["y"].to(self.device)
            outputs = self.model(inputs)
            loss = self.loss_fn(targets, outputs)
            final_loss += loss.item()
        return final_loss / len(data_loader)

class Model(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size, num_layers, dropout):
        super().__init__()
        layers = []
        for _ in range(num_layers):
            if len(layers) == 0:
                layers.append(nn.Linear(num_features, hidden_size))
                layers.append(nn.BatchNorm1d(hidden_size))
                layers.append(nn.Dropout(dropout))
                layers.append(nn.Linear(hidden_size, hidden_size))
                layers.append(nn.BatchNorm1d(hidden_size))
                layers.append(nn.Dropout(dropout))
                layers.append(nn.ReLU())
            else:
                layers.append(nn.Linear(hidden_size, hidden_size))
                layers.append(nn.BatchNorm1d(hidden_size))
                layers.append(nn.Dropout(dropout))
                
                layers.append(nn.Linear(hidden_size, hidden_size))
                layers.append(nn.BatchNorm1d(hidden_size))
                layers.append(nn.Dropout(dropout))
                layers.append(nn.ReLU())

        layers.append(nn.Linear(hidden_size, num_targets))
        self.model = nn.Sequential(*layers)   # Layers in the list

    def forward(self, x):
        return self.model(x)

In [6]:

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
EPOCS = 100


def run_training(fold, params, save_model=False):
    df = pd.read_csv('/content/train_features.csv')
    df = df.drop(["cp_type", "cp_time", "cp_dose"], axis=1)
    targets_df = pd.read_csv("/content/train_targets_fold.csv")

    feature_cols = df.drop("sig_id", axis=1).columns
    target_cols = targets_df.drop(["sig_id", "kfold"], axis=1).columns

    df = df.merge(targets_df, on="sig_id", how="left")

    train_df = df[df.kfold != fold].reset_index(drop=True)
    valid_df = df[df.kfold == fold].reset_index(drop=True)

    xtrain = train_df[feature_cols].to_numpy()
    ytrain = train_df[target_cols].to_numpy()

    xvalid = valid_df[feature_cols].to_numpy()
    yvalid = valid_df[target_cols].to_numpy()

    train_dataset = MoaDataset(features=xtrain, targets=ytrain)
    valid_dataset = MoaDataset(features=xvalid, targets=yvalid)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=1024, num_workers=8, shuffle=True
    )

    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=1024, num_workers=8
    )

    model = Model(
        num_features=xtrain.shape[1],
        num_targets=ytrain.shape[1],
        num_layers=params["num_layers"],
        hidden_size=params["hidden_size"],
        dropout=params["dropout"]
    )

    model.to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=params["learning_rate"])
    eng = Engine(model, optimizer=optimizer, device=DEVICE)

    best_loss = np.inf
    early_stopping_iter = 10
    early_stopping_counter = 0

    for epoch in range(EPOCS):
        train_loss = eng.train(train_loader)
        valid_loss = eng.evaluate(valid_loader)
        print(f"FOLD : {fold}, EPOCH : {epoch}, TRAIN_LOSS : {train_loss}, VALIDATION_LOSS : {valid_loss}")
        if valid_loss < best_loss:
            best_loss = valid_loss
            if save_model:
                torch.save(model.state_dict(), f"Model_{fold}.bin")
        else:
            early_stopping_counter += 1

        if early_stopping_counter > early_stopping_iter:
            break
    return best_loss


# Optuna functions
def objective(trial):
    params = {
        "num_layers": trial.suggest_int("num_layers", 1, 7),
        "hidden_size": trial.suggest_int("hidden_size", 16, 2048),
        "dropout": trial.suggest_uniform("dropout", 0.1, 0.7),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-6, 1e-3)
    }
    all_loss = []
    for f in range(5):
        temp_loss = run_training(f, params, save_model=False)
        all_loss.append(temp_loss)

    return np.mean(all_loss)


if __name__ == "__main__":
    study =optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=4)

    print("Best trial: ")
    trial_ = study.best_trial

    print(trial_.values)
    print(trial_.params)
    scores = 0
    for j in range(5):
        scr = run_training(j, trial_.params, save_model=True)
        scores += scr

    print(scores / 5)

[I 2021-11-01 13:30:41,217] A new study created in memory with name: no-name-510bb59c-5bb7-4bfa-8553-582b5870b4ee
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning:

This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



FOLD : 0, EPOCH : 0, TRAIN_LOSS : 0.4040809022752862, VALIDATION_LOSS : 0.3319945454597473
FOLD : 0, EPOCH : 1, TRAIN_LOSS : 0.10106267446750089, VALIDATION_LOSS : 0.026108254119753836
FOLD : 0, EPOCH : 2, TRAIN_LOSS : 0.038867049213302765, VALIDATION_LOSS : 0.02504754178225994
FOLD : 0, EPOCH : 3, TRAIN_LOSS : 0.02663332488583891, VALIDATION_LOSS : 0.021101005002856256
FOLD : 0, EPOCH : 4, TRAIN_LOSS : 0.023859893903136253, VALIDATION_LOSS : 0.022065461799502372
FOLD : 0, EPOCH : 5, TRAIN_LOSS : 0.022614449556720883, VALIDATION_LOSS : 0.021850796416401863
FOLD : 0, EPOCH : 6, TRAIN_LOSS : 0.02179811649808758, VALIDATION_LOSS : 0.02145155519247055
FOLD : 0, EPOCH : 7, TRAIN_LOSS : 0.021151708537026456, VALIDATION_LOSS : 0.020864711701869966
FOLD : 0, EPOCH : 8, TRAIN_LOSS : 0.020715935645919097, VALIDATION_LOSS : 0.02048543095588684
FOLD : 0, EPOCH : 9, TRAIN_LOSS : 0.020340132085900558, VALIDATION_LOSS : 0.020109176635742188
FOLD : 0, EPOCH : 10, TRAIN_LOSS : 0.020020980485960058, VAL

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning:

This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



FOLD : 1, EPOCH : 0, TRAIN_LOSS : 0.3974768346861789, VALIDATION_LOSS : 0.34213905334472655
FOLD : 1, EPOCH : 1, TRAIN_LOSS : 0.09994569009071902, VALIDATION_LOSS : 0.026289477199316024
FOLD : 1, EPOCH : 2, TRAIN_LOSS : 0.038999615237116814, VALIDATION_LOSS : 0.024790650606155394
FOLD : 1, EPOCH : 3, TRAIN_LOSS : 0.026557885894649906, VALIDATION_LOSS : 0.02087402492761612
FOLD : 1, EPOCH : 4, TRAIN_LOSS : 0.023736826487277683, VALIDATION_LOSS : 0.021731382980942727
FOLD : 1, EPOCH : 5, TRAIN_LOSS : 0.02249982668773124, VALIDATION_LOSS : 0.021721676364541052
FOLD : 1, EPOCH : 6, TRAIN_LOSS : 0.021675664637433856, VALIDATION_LOSS : 0.021343250572681428
FOLD : 1, EPOCH : 7, TRAIN_LOSS : 0.02110685202244081, VALIDATION_LOSS : 0.020780109986662864
FOLD : 1, EPOCH : 8, TRAIN_LOSS : 0.02065186584858518, VALIDATION_LOSS : 0.020426512509584428
FOLD : 1, EPOCH : 9, TRAIN_LOSS : 0.02030294261088497, VALIDATION_LOSS : 0.020069700479507447
FOLD : 1, EPOCH : 10, TRAIN_LOSS : 0.02003007970358196, VAL

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning:

This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



...

FOLD : 2, EPOCH : 0, TRAIN_LOSS : 0.15632270472614387, VALIDATION_LOSS : 0.17744550108909607
FOLD : 2, EPOCH : 1, TRAIN_LOSS : 0.022731197311689978, VALIDATION_LOSS : 0.07453968673944474
FOLD : 2, EPOCH : 2, TRAIN_LOSS : 0.02137107627564355, VALIDATION_LOSS : 0.08487599641084671
FOLD : 2, EPOCH : 3, TRAIN_LOSS : 0.020549028621692406, VALIDATION_LOSS : 0.05417899563908577
FOLD : 2, EPOCH : 4, TRAIN_LOSS : 0.0200641268962308, VALIDATION_LOSS : 0.058543752133846286
FOLD : 2, EPOCH : 5, TRAIN_LOSS : 0.019748192966768618, VALIDATION_LOSS : 0.06119646802544594
FOLD : 2, EPOCH : 6, TRAIN_LOSS : 0.01946859395033435, VALIDATION_LOSS : 0.055536966770887375
FOLD : 2, EPOCH : 7, TRAIN_LOSS : 0.0191166019557338, VALIDATION_LOSS : 0.04772274121642113
FOLD : 2, EPOCH : 8, TRAIN_LOSS : 0.018816347969205754, VALIDATION_LOSS : 0.04568613916635513
FOLD : 2, EPOCH : 9, TRAIN_LOSS : 0.018504781060312923, VALIDATION_LOSS : 0.036514962464571
FOLD : 2, EPOCH : 10, TRAIN_LOSS : 0.01827210668278368, VALIDATION_

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning:

This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



FOLD : 3, EPOCH : 0, TRAIN_LOSS : 0.15870020430731147, VALIDATION_LOSS : 0.19400401711463927
FOLD : 3, EPOCH : 1, TRAIN_LOSS : 0.0227673728215067, VALIDATION_LOSS : 0.07523739486932754
FOLD : 3, EPOCH : 2, TRAIN_LOSS : 0.02123340915300344, VALIDATION_LOSS : 0.0754309356212616
FOLD : 3, EPOCH : 3, TRAIN_LOSS : 0.02047009766101837, VALIDATION_LOSS : 0.05876034274697304
FOLD : 3, EPOCH : 4, TRAIN_LOSS : 0.02007108524833855, VALIDATION_LOSS : 0.04700796976685524
FOLD : 3, EPOCH : 5, TRAIN_LOSS : 0.01974183330802541, VALIDATION_LOSS : 0.04910018220543862
FOLD : 3, EPOCH : 6, TRAIN_LOSS : 0.019328253245667407, VALIDATION_LOSS : 0.049600125104188916
FOLD : 3, EPOCH : 7, TRAIN_LOSS : 0.019065940360489645, VALIDATION_LOSS : 0.04316105768084526
FOLD : 3, EPOCH : 8, TRAIN_LOSS : 0.018784207929121822, VALIDATION_LOSS : 0.039908699691295624
FOLD : 3, EPOCH : 9, TRAIN_LOSS : 0.01842432704411055, VALIDATION_LOSS : 0.03450139462947845
FOLD : 3, EPOCH : 10, TRAIN_LOSS : 0.018256490854056256, VALIDATION

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning:

This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



FOLD : 4, EPOCH : 0, TRAIN_LOSS : 0.15769445327551743, VALIDATION_LOSS : 0.22202714383602143
FOLD : 4, EPOCH : 1, TRAIN_LOSS : 0.022735428947367166, VALIDATION_LOSS : 0.09433613121509551
FOLD : 4, EPOCH : 2, TRAIN_LOSS : 0.021278662999209604, VALIDATION_LOSS : 0.07304844111204148
FOLD : 4, EPOCH : 3, TRAIN_LOSS : 0.020517781572906596, VALIDATION_LOSS : 0.050241556763648984
FOLD : 4, EPOCH : 4, TRAIN_LOSS : 0.02004499106030715, VALIDATION_LOSS : 0.05904196947813034
FOLD : 4, EPOCH : 5, TRAIN_LOSS : 0.01977852888797459, VALIDATION_LOSS : 0.05036391094326973
FOLD : 4, EPOCH : 6, TRAIN_LOSS : 0.01939354680086437, VALIDATION_LOSS : 0.04645088762044906
FOLD : 4, EPOCH : 7, TRAIN_LOSS : 0.019036975934317236, VALIDATION_LOSS : 0.03880562409758568
FOLD : 4, EPOCH : 8, TRAIN_LOSS : 0.018743264047723068, VALIDATION_LOSS : 0.0354189045727253
FOLD : 4, EPOCH : 9, TRAIN_LOSS : 0.018479825634705394, VALIDATION_LOSS : 0.03321818709373474
FOLD : 4, EPOCH : 10, TRAIN_LOSS : 0.018148611642812427, VALIDAT